In [47]:
# 1.目的
# 予想脚質がゴール直近の通貨順位に対してどのくらい正確なのか調べたい
# （正確なのであれば、例えば予想脚質が"逃げ"ならゴール直前の通貨順位は小さいはず）
# 2.やったこと
# XGBoostを使う。目的変数を通過順位とし、説明変数を予想脚質として予測したときの精度を調べる
# 3.考察
# 例えば予想脚質が追込である馬が8頭中6頭いて6頭が後方にいたとしても、前から追込で3番目の馬がいることになる。
# この場合を考慮すると予想の精度が低いと判断するのはおかしいので、本分析のやり方を見直した方がよさそう。

In [3]:
# インポート
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import xgboost as xgb
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score, precision_score, recall_score, f1_score

In [9]:
# CSVファイルの読み込み
# df = pd.read_csv('input3.csv', encoding='cp932')
df = pd.read_csv('input3.csv', encoding='utf-8')

In [23]:
# style_nameはカテゴリ変数であり文字列だがそのまま使えないのでエンコーディングして数値に変換する

In [27]:
import pandas as pd

# CSVファイルを読み込み（元のデータは変更しない）
df = pd.read_csv('input3.csv', encoding='utf-8')

# 明示的にマッピングする辞書を定義
style_mapping = {
    '逃げ': 1,
    '先行': 2,
    '差し': 3,
    '追込': 4
}
# 新しい列 'style_encoded' を追加（元の 'style_name' 列はそのまま）
df['style_encoded'] = df['style_name'].map(style_mapping)

In [29]:
# 回帰の場合→XGBRegressorを使う

In [39]:
# 目的変数をposition_1にしてみる

In [45]:
# 説明変数
x = df[["style_encoded"]]
# 目的変数
y1 = df["position_1"]
y2 = df["position_2"]
y3 = df["position_3"]
# 学習データと検証データの分割（検証データ割合が0.500669344）
x_train, x_test, y1_train, y1_test = train_test_split(x, y1, train_size=0.500669344, shuffle=False)
x_train, x_test, y2_train, y2_test = train_test_split(x, y2, train_size=0.500669344, shuffle=False)
x_train, x_test, y3_train, y3_test = train_test_split(x, y3, train_size=0.500669344, shuffle=False)
# モデルの学習
model1 = xgb.XGBRegressor(objective='reg:squarederror', max_depth=3, random_state=42)
model1.fit(x_train, y1_train)
model2 = xgb.XGBRegressor(objective='reg:squarederror', max_depth=3, random_state=42)
model2.fit(x_train, y2_train)
model3 = xgb.XGBRegressor(objective='reg:squarederror', max_depth=3, random_state=42)
model3.fit(x_train, y3_train)
# 予測
y1_pred = model1.predict(x_test)
y2_pred = model2.predict(x_test)
y3_pred = model3.predict(x_test)
# 評価
rmse1 = np.sqrt(mean_squared_error(y1_test, y1_pred))
r2_1 = r2_score(y1_test, y1_pred)
print(f"RMSE: {rmse1:.4f}")
print(f"R^2 Score: {r2_1:.4f}")
rmse2 = np.sqrt(mean_squared_error(y2_test, y2_pred))
r2_2 = r2_score(y2_test, y2_pred)
print(f"RMSE: {rmse2:.4f}")
print(f"R^2 Score: {r2_2:.4f}")
rmse3 = np.sqrt(mean_squared_error(y3_test, y3_pred))
r2_3 = r2_score(y3_test, y3_pred)
print(f"RMSE: {rmse3:.4f}")
print(f"R^2 Score: {r2_3:.4f}")

RMSE: 2.0136
R^2 Score: 0.2265
RMSE: 2.0154
R^2 Score: 0.2247
RMSE: 2.0676
R^2 Score: 0.1927
